In [1]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
import pydot_ng
%matplotlib inline

from sklearn.datasets import make_moons

#### Installing Theano

Many people can just:

```
pip install theano
```

And it should work.  I had trouble on my Windows machine, but I'm told that is not always the case.  You may need a C compiler for it to work, though I've heard you should be able to just get around it (even using the default settings).  There's a couple of other things you may consider installing:

* `pip install pydot` and [graphviz](http://www.graphviz.org/) for fancy visualizations
* CUDA drivers and whatever else is necessary if you want to run this on your GPU.

# Basic Theano commands

In [2]:
x1 = T.scalar()
w1 = T.scalar()
w0 = T.scalar()
w4 = T.scalar()
z1 = w1 * x1 + w0*w4

In [3]:
f = theano.function([w1,x1,w0, w4], z1)

ImportError: DLL load failed: A dynamic link library (DLL) initialization routine failed.

In [ ]:
f(2, 3, 4, 16)

In [ ]:
theano.printing.pydotprint(f, outfile="simple_graph.png", var_with_name_simple=True)

<img src="./simple_graph.png"/>

# A feedforward neural network in Theano

A 2-layer feedforward neural network using Theano. 

Parameters:
 * `n_inputs` -- number of nodes in the input layer
 * `n_hidden` -- number of nodes in the hidden layer
 * `n_outputs` -- number of nodes in the hidden layer
 * `epochs` -- number of training epochs
 * `print_every` -- on which epoch should the current cost print
 * `n_samples` -- number of training samples
 * `reg` -- regularization strength
 * `alpha` -- learning rate

In [ ]:
# Set paramters
(n_inputs, n_hidden, n_outputs, epochs, print_every,
    n_samples, batch, reg, alpha) = (2, 4, 1, 1000, 100, 200, 20, 0.01, 0.01)

Let's build the neural network:

In [ ]:
# Need to initialize the parameters to a small, random number
noise = 1/(np.sqrt(n_inputs * n_outputs * n_hidden))

# Weights and biases are theano shared variables, so that they can hold a state
W1 = theano.shared(noise * np.random.randn(n_inputs, n_hidden), name='W1')
W2 = theano.shared(noise * np.random.randn(n_hidden, n_outputs), name='W2')
b1 = theano.shared(np.zeros(n_hidden), name='b1')
b2 = theano.shared(np.zeros(n_outputs), name='b2')

x = T.dmatrix('x')
y = T.dvector('y')

# forward prop
z1 = x.dot(W1) + b1
hidden = T.tanh(z1)
z2 = hidden.dot(W2) + b2
output = T.nnet.sigmoid(z2)
prediction = output > 0.5

# cost function
crossent = -y.dot(T.log(output)) - (1 - y).dot(T.log(1-output))
cost = crossent.sum() + reg * ((W1**2).sum() + (W2**2).sum())

# gradients
gW1, gb1, gW2, gb2 = T.grad(cost, [W1, b1, W2, b2])

# build theano functions
epoch = theano.function(inputs = [x, y],
                        outputs = [output, crossent.sum()],
                        updates = ((W1, W1 - alpha * gW1),
                                   (b1, b1 - alpha * gb1),
                                   (W2, W2 - alpha * gW2),
                                   (b2, b2 - alpha * gb2)))
predict = theano.function(inputs=[x], outputs=prediction)

Now that we've built the network, let's generate some toy data and work with it.

In [ ]:
# generate toy data
data = make_moons(n_samples=n_samples, noise=.17, random_state=7)

plt.scatter(data[0][:,0], data[0][:,1], c=data[1], cmap='viridis', s=40);

In [ ]:
# train the model
for i in range(epochs):
    pred,err = epoch(data[0], data[1])
    if i % print_every == 0:
        print('Error after epoch {}: {}'.format(i, err))

# check accuracy
preds = predict(data[0]).T[0]
wrong = (preds != data[1]).sum()
score = (n_samples * 1.0 - wrong) / n_samples
print("Our model made {} errors, for an accuracy of {}".format(wrong, score))

In [ ]:
fig = plt.figure(figsize=(12, 8))

# Plot decision bounary
x0_min, x0_max = data[0][:,0].min(), data[0][:,0].max()
x1_min, x1_max = data[0][:,1].min(), data[0][:,1].max()

xx0, xx1 = np.meshgrid(np.linspace(x0_min, x0_max, 1000),
                       np.linspace(x1_min, x1_max, 1000))

grid_flat = np.stack([xx0.ravel(), xx1.ravel()], axis=1)

yy = predict(grid_flat).T[0].reshape(xx0.shape)

plt.contourf(xx0, xx1, yy, levels=[0,.5,1], alpha=0.5, cmap='viridis')

# Plot data
plt.scatter(data[0][:,0], data[0][:,1], c=data[1], cmap='viridis', s=40);

Finally, let's view the graph of our neural network.

In [ ]:
theano.printing.pydotprint(predict, outfile="ffnn.png", var_with_name_simple=True)

<img src='./ffnn.png' />